In [ ]:
#!pip install scikit-learn

In [ ]:
#!pip install pandas

In [ ]:
#!pip install matplotlib

In [ ]:
#!pip install numpy

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pylab
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

#Добавил библиотеки
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [ ]:
data = pd.read_csv('/datasets/Churn.csv')

In [ ]:
data.info()

In [ ]:
data.head(10)

**Соотношение классов:**

**Доля клиентов, покинувших банк:**

In [ ]:
sum(data['Exited']) / len(data)

**Доля оставшихся клиентов:**

In [ ]:
1 - sum(data['Exited']) / len(data)

Имеет место значительный дисбаланс классов, что нужно будет учесть в дальнейшей работе.

In [ ]:
data['Exited'].value_counts(normalize=True)

**Удаление лишних столбцов:**

Перед началом обучения нужно также избавиться от лишних столбцов.

In [ ]:
print(f"Количество столбцов до операции: {data.shape[1]}")
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
f"Количество столбцов после операции: {data.shape[1]}"

**Заполнение пропусков:**

При вызове data.info() увидели 909 пропусков в столбце Tenure. Заполним их. 

In [ ]:
data['Tenure'].value_counts().plot(kind='bar')
print(data['Tenure'].median())
data['Tenure'].fillna(data['Tenure'].median(), inplace=True)

**Выделение признаков, разделение на выборки:**

Выделим признаки и целевой признак

In [ ]:
features = data.drop(columns=['Exited'])
target = data['Exited']

Разделим на обучающую, валидационную и тестовую выборки

In [ ]:
features_train, features_mixed, target_train, target_mixed = train_test_split(features, target, test_size = 0.4,
                                                                              random_state=31415)

features_valid, features_test, target_valid, target_test = train_test_split(features_mixed, target_mixed, test_size = 0.5,
                                                                            random_state=31415)
print('Размеры выборок')
f"Обучающая {len(features_train)}, валидационная {len(features_valid)}, тестовая {len(features_test)}"

**OneHotEncoding**

In [ ]:
chosen_columns=['Gender','Geography']
enc = OneHotEncoder(drop='first', sparse=False).fit(features_train[chosen_columns])
#Тренировочная выборка
features_train_encoded = pd.DataFrame(enc.transform(features_train[chosen_columns]),
                                      index=features_train.index, columns=['Male', 'Germany', 'Spain'])

features_train = pd.concat([features_train, features_train_encoded], axis=1).drop(columns=chosen_columns)


#Валидационная выборка
features_valid_encoded = pd.DataFrame(enc.transform(features_valid[chosen_columns]),
                                      index=features_valid.index, columns=['Male', 'Germany', 'Spain'])

features_valid = pd.concat([features_valid, features_valid_encoded], axis=1).drop(columns=chosen_columns)


#Тестовая выборка
features_test_encoded = pd.DataFrame(enc.transform(features_test[chosen_columns]),
                                      index=features_test.index, columns=['Male', 'Germany', 'Spain'])

features_test = pd.concat([features_test, features_test_encoded], axis=1).drop(columns=chosen_columns)

In [ ]:
features_train.head()

**Масштабирование**

In [ ]:
pd.options.mode.chained_assignment = None
numeric = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'Tenure', 'NumOfProducts']
scaler = StandardScaler()

scaler.fit(features_train[['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'Tenure', 'NumOfProducts']])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

## Исследование задачи

Начнём обучать модели и изучать метрики без борьбы с дисбалансом классов.

**Решающее дерево**

In [ ]:
##depth_dtc_plot = []
##f1_dtc_plot = []
##
##best_f1 = 0
##best_depth = 0
##for depth in range(1, 20):
##    model = DecisionTreeClassifier(random_state=31415, max_depth=depth)
##    model.fit(features_train, target_train)
##    predicted_valid = model.predict(features_valid)
##    f1 = f1_score(predicted_valid, target_valid)
##    depth_dtc_plot.append(depth)
##    f1_dtc_plot.append(f1)
##    if f1 > best_f1:
##        best_f1 = f1
##        best_depth = depth
##        
##dtc = DecisionTreeClassifier(max_depth = best_depth, random_state=31415)
##dtc.fit(features_train, target_train)
##probabilities_dtc = dtc.predict_proba(features_train)
##probabilities_one_dtc = probabilities_dtc[:, 1]
##predictions_dtc = dtc.predict(features_valid)
##
##f"Accuracy DTC = {accuracy_score(target_valid, predictions_dtc)}, F1 DTC = {f1_score(target_valid, predictions_dtc)}, AUC-ROC = {roc_auc_score(target_train, probabilities_one_dtc)}"

In [ ]:
###Параметры графика
##plt.figure(figsize=(8, 3))
##plt.step(depth_dtc_plot, f1_dtc_plot)
##plt.title('Зависимость F1 от глубины в DTC')
##plt.xlabel('Глубина')
##plt.ylabel('F1')
##pylab.xticks(range(20))
##plt.grid()
##plt.show()

**Случайный лес**

Сначала найдём оптимальную глубину при фиксированном числе деревьев

In [ ]:
depth_rfc_plot = []
f1_rfc_plot = []

best_f1 = 0
best_depth = 0
for depth in range(1, 21):
    model = RandomForestClassifier(random_state=31415, max_depth = depth)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    depth_rfc_plot.append(depth)
    f1_rfc_plot.append(f1)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
        
        
rfc = RandomForestClassifier(random_state=31415, max_depth=best_depth)
rfc.fit(features_train, target_train)
probabilities_rfc = rfc.predict_proba(features_valid)
probabilities_one_rfc = probabilities_rfc[:, 1]
f"Лучшая f1 {best_f1}, лучшая глубина {best_depth}, AUC-ROC = {roc_auc_score(target_valid, probabilities_one_rfc)}"

In [ ]:
plt.figure(figsize=(8, 3))
plt.step(depth_rfc_plot, f1_rfc_plot)
plt.title('Зависимость F1 от глубины в RFC')
plt.xlabel('Глубина')
plt.ylabel('F1')
pylab.xticks(range(21))
plt.grid()
plt.show()

Теперь найдём оптимальное число деревьев при этой глубине

In [ ]:
est_rfc_plot = []
f1_rfc_plot = []

best_est = 0
best_f1 = 0
for est in range(100, 121):
    model = RandomForestClassifier(random_state=31415, max_depth=17, n_estimators=est)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    est_rfc_plot.append(est)
    f1_rfc_plot.append(f1)
    if f1 > best_f1:
        best_f1 = f1
        best_est = est
        
rfc = RandomForestClassifier(random_state=31415, max_depth=best_depth, n_estimators=best_est)
rfc.fit(features_train, target_train)
probabilities_rfc = rfc.predict_proba(features_valid)
probabilities_one_rfc = probabilities_rfc[:, 1]
f"F1 RFC = {best_f1}, Лучшее количество деревьев = {best_est}, AUC-ROC = {roc_auc_score(target_valid, probabilities_one_rfc)}"

In [ ]:
#Параметры графика
plt.figure(figsize=(8, 3))
plt.step(est_rfc_plot, f1_rfc_plot)
plt.title('Зависимость F1 от n_estimators')
plt.xlabel('n_estimators')
plt.ylabel('F1')
plt.grid()
plt.show()

**Логистическая регрессия**

In [ ]:
##iter_lr_plot = []
##f1_lr_plot = []
##
##best_f1 = 0
##best_iter = 0
##for itera in range (100, 2001, 100):
##    model = LogisticRegression(max_iter=itera, solver='liblinear', random_state=31415)
##    model.fit(features_train, target_train)
##    predicted_valid = model.predict(features_valid)
##    f1 = f1_score(target_valid, predicted_valid)
##    iter_lr_plot.append(itera)
##    f1_lr_plot.append(f1)
##    if f1 > best_f1:
##        best_f1 = f1
##        best_iter = itera
##        
##lr = LogisticRegression(random_state=31415, solver='liblinear', max_iter = best_iter)
##lr.fit(features_train, target_train)
##probabilities_lr = lr.predict_proba(features_train)
##probabilities_one_lr = probabilities_lr[:, 1]
##f"F1 LR = {best_f1}, Лучшее значение max_iterations = {best_est}, AUC-ROC = {roc_auc_score(target_train, probabilities_one_lr)}"

In [ ]:
###Параметры графика
##plt.figure(figsize=(8, 3))
##plt.step(iter_lr_plot, f1_lr_plot)
##plt.title('Зависимость F1 от max_iterations')
##plt.xlabel('max_iterations')
##plt.ylabel('F1')
##plt.grid()
##plt.show()

**Выводы:**
<br>На валидационной выборке без учёта дизбаланса классов, лучшую F1-меру (0.55) показывает случайный лес с гиперпараметрами max_depth = 9, n_estimators = 10. 
<br>UPD: Видим, что и лучший AUC-ROC показывает случайный лес. При этом, изменение гиперпараметрво немного повлияло на F1, но AUC-ROC остался примерно таким же.

## Борьба с дисбалансом

 **UPSAMPLING**
<BR>В первую очередь воспользуемся upsampling'ом, и посмотрим, как изменится F1 мера нашей модели

In [ ]:
#Создаём функцию upsample
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=31415)
    
    return features_upsampled, target_upsampled

In [ ]:
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4)
#features_valid_upsampled, target_valid_upsampled = upsample(features_valid, target_valid, 4)

In [ ]:
model_upsample_test = RandomForestClassifier(random_state=31415, max_depth=17, n_estimators=106)
model_upsample_test.fit(features_train_upsampled, target_train_upsampled)
predicted_valid_upsampled = model_upsample_test.predict(features_valid)
probabilities_ut = model_upsample_test.predict_proba(features_valid)
probabilities_one_ut = probabilities_ut[:, 1]
f"Accuracy модели = {accuracy_score(target_valid, predicted_valid_upsampled)}, F1-мера = {f1_score(target_valid, predicted_valid_upsampled)}, AUC-ROC = {roc_auc_score(target_valid, probabilities_one_ut)}"


Видим рост F1 с 0.58 до 0.587 на валидационной выборке. Переходим к Downsampling'у

**DOWNSAMPLING**

In [ ]:
#Создаём функцию downsample
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=31415)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=31415)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=31415)
    
    return features_downsampled, target_downsampled

In [ ]:
features_train_downsampled, target_train_downsampled = downsample(features_train, target_train, 0.25)
#features_valid_downsampled, target_valid_downsampled = downsample(features_valid, target_valid, 0.25)

In [ ]:
model_downsample_test = RandomForestClassifier(random_state=31415, max_depth=17, n_estimators=106)
model_downsample_test.fit(features_train_downsampled, target_train_downsampled)
predicted_valid_downsampled = model_downsample_test.predict(features_valid)
probabilities_dt = model_downsample_test.predict_proba(features_valid)
probabilities_one_dt = probabilities_dt[:, 1]
f"Accuracy модели = {accuracy_score(target_valid, predicted_valid_downsampled)}, F1-мера = {f1_score(target_valid, predicted_valid_downsampled)}, AUC-ROC = {roc_auc_score(target_valid, probabilities_one_dt)}"

Получаем лучшую на данный момент F1 меру на валидационной выборке - **0.615**. UPD: При этом видим, что AUC-ROC в обоих случаях примерно одинаковый, и downsampling даже показывает немного худший результат

## Тестирование модели

**F1 - мера**

In [ ]:
model = RandomForestClassifier(random_state=31414, max_depth=17, n_estimators=106)
model.fit(features_train_upsampled, target_train_upsampled)
predicted_valid = model.predict(features_test)
f"F1 = {f1_score(target_test, predicted_valid)}"

**AUC-ROC**

In [ ]:
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

f"AUC-ROC = {roc_auc_score(target_test, probabilities_one_valid)}"

In [ ]:
fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_valid)
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC-кривая')
plt.show()

**Вывод по шагу:**
<br>Итого, после испытания разных способов борьбы с дисбалансом классов, видим лучшие показатели при использовании downsampling'a. Итого, после тестов на тестовой выборке, получаем AUC-ROC равный 0.852 и F1 = 0.571. 

**Итого:**
<br>Прежде всего для обучения моделей и построения прогнозов, данные было нужно обработать. Количестенные столбцы были масштабированы, категориальные обработаны техникой OHE. В процессе ознакомления с данными был выявлен дисбаланас классов (4 к 1), который впоследствии был также обработан. Также в данных обнаружились пропуски, (суммарно 909 значений), которые были заполнены медианным значением.
<br>
<br>После теста разных моделей (dtc, rfc и lr) на тренировочных данных, лучшим образом себя показал случайный лес, выдав наивысшую F1 меру. После перебора гиперпараметров, лучшими значениями были признаны max_depth = 17 и n_estimators = 106. Однако перед обучением финальной модели всё ещё нужно было решить проблему несбалансированности классов.
<br>
<br>Были изучены показатели модели при использовании downsapling'a и upsampling'a, но с отрывом на 0.02 пункта лучше себя показал downsampling. По результатам обучения модели на полученных гиперпараметрах, получили AUC-ROC, равный 0.85 и F1 меру, равную 0.59.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*